**1. Análise da sequência e das features presentes no NCBI**

In [1]:
from Bio import Entrez
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqFeature

In [3]:
Entrez.email = 'pg40082@alunos.uminho.pt'

In [5]:
#Neste script os genes serão manipulados na seguinte ordem: glgA, glgB, glgD e SO785_RS05220
list_nomes_genes = ['glgA', 'glgB', 'glgD', 'SO785_RS05220']
list_genes_id = ['93290189', '93290192', '93290190', '93290188,']
list_genomic_sequence = 'NZ_CP139575.1' #igual para os 4 genes
list_mRNA_id = ['WP_003546574.1 ', 'WP_003546573.1', 'WP_003546577.1 '] #glgB não tem
list_protein_id = ['Q5FL65','Q5FL68','Q5FL66', 'Q5FL64'] 

## **Análise das sequências, características dos genes e das proteínas**

### **Proteínas**

In [13]:
protein_id = ['Q5FL65', 'Q5FL68', 'Q5FL66', 'Q5FL64']

# Mapeamento entre IDs das proteínas e nomes dos genes
protein_gene_map = {
    'Q5FL65.1': 'glgA',
    'Q5FL68.1': 'glgB',
    'Q5FL66.1': 'glgD',  
    'Q5FL64.1': 'SO785_RS05220'
}


def fetch_protein_records(protein_id):
    records = [] 
    for protein in protein_id:
        try:
            with Entrez.efetch(db="protein", id=protein, rettype="gb", retmode="text") as handle:
                record = SeqIO.read(handle, "genbank")
                records.append(record)
        except Exception as e:
            print(f"Erro ao processar o ID da proteína {protein}: {e}")
    return records


records = fetch_protein_records(protein_id)


for record in records:
    gene_name = protein_gene_map.get(record.id, "Desconhecido")
    
    print(f"--- Informações sobre o Gene e a Proteína ---")
    print(f"Nome do Gene: {gene_name}")
    print(f"ID da proteína: {record.id}")
    print(f"Descrição da proteína: {record.description}")
    
    organism = record.annotations.get("organism", "Desconhecido")
    taxonomy = " > ".join(record.annotations.get("taxonomy", ["Desconhecido"]))
    print(f"Organismo: {organism}")
    print(f"Taxonomia: {taxonomy}")
    
    print("Domínios Proteicos:")
    for feature in record.features:
        if feature.type in ['region', 'CDS', 'exon', 'repeat_region']:
            start = feature.location.start
            end = feature.location.end
            print(f"  - Tipo: {feature.type} | Localização: {start} até {end}")
            
            if "product" in feature.qualifiers:
                print(f"    Produto: {', '.join(feature.qualifiers['product'])}")
            if "note" in feature.qualifiers:
                print(f"    Nota: {', '.join(feature.qualifiers['note'])}")
            if "db_xref" in feature.qualifiers:
                print(f"    Referências externas: {', '.join(feature.qualifiers['db_xref'])}")
    
    print(f"Tamanho da Sequência: {len(record.seq)} aminoácidos")
    print(f"Sequência (primeiros 50 aa): {record.seq[:50]}...\n")

--- Informações sobre o Gene e a Proteína ---
Nome do Gene: glgA
ID da proteína: Q5FL65.1
Descrição da proteína: RecName: Full=Glycogen synthase; AltName: Full=Starch [bacterial glycogen] synthase
Organismo: Lactobacillus acidophilus NCFM
Taxonomia: Bacteria > Bacillati > Bacillota > Bacilli > Lactobacillales > Lactobacillaceae > Lactobacillus
Domínios Proteicos:
Tamanho da Sequência: 476 aminoácidos
Sequência (primeiros 50 aa): MRVLFTGAECAPFFKTGGLGDVLGSLPNQLAKEGVDVGVVLPLYQDLPEK...

--- Informações sobre o Gene e a Proteína ---
Nome do Gene: glgB
ID da proteína: Q5FL68.1
Descrição da proteína: RecName: Full=1,4-alpha-glucan branching enzyme GlgB; AltName: Full=1,4-alpha-D-glucan:1,4-alpha-D-glucan 6-glucosyl-transferase; AltName: Full=Alpha-(1->4)-glucan branching enzyme; AltName: Full=Glycogen branching enzyme; Short=BE
Organismo: Lactobacillus acidophilus NCFM
Taxonomia: Bacteria > Bacillati > Bacillota > Bacilli > Lactobacillales > Lactobacillaceae > Lactobacillus
Domínios Proteicos

### **mRNA**

In [16]:
mRNA_gene_map = {
    'WP_003546574.1': 'glgA',
    'WP_003546573.1': 'glgD',
    'WP_003546577.1': 'SO785_RS05220'
}

def fetch_records(mRNA_id):
    records = [] 
    for mRNA in mRNA_id:
        try:
            with Entrez.efetch(db="protein", id=mRNA, rettype="gb", retmode="text") as handle:  
                record = SeqIO.read(handle, "genbank")
                records.append(record)
        except Exception as e:
            print(f"Erro ao processar mRNA ID {mRNA}: {e}")
    return records


mRNA_id = ['WP_003546574.1', 'WP_003546573.1', 'WP_003546577.1']


records = fetch_records(mRNA_id)

for record in records:
    print(f"--- Informações sobre o Gene ---")
    gene_name = mRNA_gene_map.get(record.id, "Desconhecido")  
    print(f"Nome do Gene: {gene_name}")
    print(f"ID da Proteína: {record.id}")
    print(f"Descrição: {record.description}")
    print(f"Tamanho da Sequência: {len(record.seq)} bases")
    print(f"Sequência (primeiros 50 bases): {record.seq[:50]}...\n")


--- Informações sobre o Gene ---
Nome do Gene: glgA
ID da Proteína: WP_003546574.1
Descrição: glycogen synthase GlgA [Lactobacillus acidophilus]
Tamanho da Sequência: 476 bases
Sequência (primeiros 50 bases): MRVLFTGAECAPFFKTGGLGDVLGSLPNQLAKEGVDVGVVLPLYQDLPEK...

--- Informações sobre o Gene ---
Nome do Gene: glgD
ID da Proteína: WP_003546573.1
Descrição: glucose-1-phosphate adenylyltransferase subunit GlgD [Lactobacillus acidophilus]
Tamanho da Sequência: 380 bases
Sequência (primeiros 50 bases): MKTNKMAAIFSNQHEYNELRPLTDERSLSALYFAGKYRIMDFALSSIVNA...

--- Informações sobre o Gene ---
Nome do Gene: SO785_RS05220
ID da Proteína: WP_003546577.1
Descrição: glycogen/starch/alpha-glucan phosphorylase [Lactobacillus acidophilus]
Tamanho da Sequência: 803 bases
Sequência (primeiros 50 bases): MQLTKDEFKKKLKNKVNIYFEEELDEASNSELYTALSGVVRDGYAPQWRR...



### **Genes**

### *glgA*

In [64]:
record = SeqIO.read('glgA.gb', 'genbank')

from Bio import SeqIO


record = SeqIO.read('glgA.gb', "genbank")


print(f"Sequência (primeiras 50 bases): {record.seq[:50]}...")
print(f"ID: {record.id}")
print(f"Nome: {record.name}")
print(f"Fonte: {record.annotations.get('source')}")


print("\nAnotações:")
for key, value in record.annotations.items():
    if isinstance(value, list):
        print(f"  {key}: {', '.join(map(str, value))}")
    else:
        print(f"  {key}: {value}")


print(f"\nNúmero de Features: {len(record.features)}")


print("\nDetalhes das Features:")
for feature in record.features:
    print(f"  Tipo: {feature.type}")
    print(f"    Localização: {feature.location}")
    if feature.qualifiers:
        for key, value in feature.qualifiers.items():
            print(f"    {key}: {', '.join(value)}")
    print("-" * 50)

Sequência (primeiras 50 bases): GTGCGAGTTTTATTTACTGGTGCTGAATGTGCACCTTTTTTCAAAACTGG...
ID: NZ_CP139575.1
Nome: NZ_CP139575
Fonte: Lactobacillus acidophilus

Anotações:
  molecule_type: DNA
  topology: linear
  data_file_division: CON
  date: 09-DEC-2024
  accessions: NZ_CP139575, REGION:, complement(1142999..1144429)
  sequence_version: 1
  keywords: RefSeq
  source: Lactobacillus acidophilus
  organism: Lactobacillus acidophilus
  taxonomy: Bacteria, Bacillati, Bacillota, Bacilli, Lactobacillales, Lactobacillaceae, Lactobacillus
  references: location: [0:1431]
authors: Kumar,A., Rawat,M., Kunde,Y., Davenport,K., Al-Sadi,R., Chain,P. and Ma,T.
title: The Complete Genome Sequence of Probiotic Lactobacillus acidophilus ATCC 9224 isolated from sour milk
journal: Unpublished
medline id: 
pubmed id: 
comment: 
, location: [0:1431]
authors: Kumar,A.
title: Direct Submission
journal: Submitted (30-NOV-2023) Bioscience, Los Alamos National Laboratory, Bikini Atoll Road SM 30 M888, Los Alamos, 